In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import *

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup
import csv; import requests
import time
import pandas as pd
import numpy as np
import os
import psutil

In [2]:
class kfc_web_scraper:
    def __init__(self):
        self.pick_up_css = "#app > div.kfc_view_box > div > div.__start_ordering_container > div > ul > li:nth-child(1) > a"
        self.drop_down_css = "#delivery > div > form > div > div > span > span > i"
        self.store_select_css = "ul.el-scrollbar__view.el-select-dropdown__list"
        self.drop_list_css = ".el-scrollbar__view"
        self.address_confirm_css = "html body.overflow div#app.__ishome div.kfc_view_box div.kfc_dialog div.__start_ordering_container div.__ordering_content div.tab-content div#delivery.tab-pane.active div.__pick_delivery_tab button.kfc_btn.__brand.__address_confirm_button"
        self.search_button = "#delivery > div > button.kfc_btn.__brand.__address_confirm_button"
        self.img = ".__product_item_image > a > .__product_im_img > img"
        self.name = ".__product_item_desc > a > h2"
        self.price = ".__product_item_desc > .__product_desc_price"
        self.main_menu = ".__main_menu_content"
        self.category = ".__main_content_section"
        self.items = ".__main_content_block > .__product_item:not([style])"
        self.url = "https://www.kfcmongolia.com/index"
        self.store_list_dict = ''
        self.main_Data_frame = pd.DataFrame(columns=["names", "prices", "categories", "image", "store"])
        
    def get_browser(self):
        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        options.add_argument('window-size=1000x1000')
        options.add_argument("disable-gpu")
        
        self.driver = webdriver.Chrome(
                                        os.getcwd() + '\chromedriver', 
                                        options=options,
#                                         service=Service(ChromeDriverManager().install())
        )
        self.driver.get(self.url)
        self.driver_process = psutil.Process(self.driver.service.process.pid)
        
        pick_up = self.driver.find_element(By.CSS_SELECTOR, self.pick_up_css)
        pick_up.click()
        
        drop_down = self.driver.find_element(By.CSS_SELECTOR, self.drop_down_css)
        drop_down.click()

    def get_store_dict(self):
        self.get_browser()
        
        time.sleep(2)
        stores = self.driver.find_element(By.CSS_SELECTOR, self.store_select_css).text
        store_list = stores.split('\n')
        self.store_list_dict = {i+1:s for i,s in enumerate(store_list)}
        
        self.quit_browser()

    def get_store_items(self):
        
        test = 0
        categories = 0
        
        self.get_store_dict()
        
        if len(self.store_list_dict) > 0:
            for i in range(len(self.store_list_dict)):
                item_imgs = []
                item_names = []
                item_prices = []
                item_categories = []
                item_store = []
                
                self.get_browser()
                while True:
                    try:
                        input_store_names = self.driver.find_elements(By.CSS_SELECTOR, 'li.el-select-dropdown__item')
                        input_store_names[i].click()
                        if input_store_names[i].text == "":
                            continue 
                        break
                    except:
                        continue

                print(len(input_store_names), i, input_store_names[i].text)
                
                store_name = input_store_names[i].text
                search_button = self.driver.find_element(By.CSS_SELECTOR, self.search_button)
                search_button.click()

                time.sleep(2)
                get_main_menu = self.driver.find_element(By.CSS_SELECTOR, self.main_menu)
                WebDriverWait(self.driver, 100).until(EC.presence_of_element_located((By.CSS_SELECTOR, self.category)))
                get_categories = get_main_menu.find_elements(By.CSS_SELECTOR, self.category)
                
                print("categories", get_categories)
                for j in range(len(get_categories)):
                    while True:
                        try:
                            get_items = get_categories[j].find_elements(By.CSS_SELECTOR, self.items)
                            category = get_categories[j].find_element(By.CSS_SELECTOR, 'h3').text
                            break
                        except:
                            print("sdfsdfsdfs")
                            continue
                    
                    for k in range(len(get_items)):
                        img = get_items[k].find_element(By.CSS_SELECTOR, self.img).get_attribute("src")
                        name = get_items[k].find_element(By.CSS_SELECTOR, self.name).text
                        price = get_items[k].find_element(By.CSS_SELECTOR, self.price).text

                        if name != '':
                            item_imgs.append(img)
                            item_categories.append(category)
                            item_names.append(name)
                            item_prices.append(price)
                            item_store.append(store_name)

                self.quit_browser()
                    
                temp_DataFrame = pd.DataFrame({"names": item_names, 
                                               "prices": item_prices,
                                               "categories": item_categories,
                                               "image": item_imgs, 
                                               "store": item_store
                                              })
                self.main_Data_frame = pd.concat([self.main_Data_frame, temp_DataFrame])
            
        return self.main_Data_frame
            
    def quit_browser(self, ):
        self.driver.quit()

In [ ]:
test = kfc_web_scraper()
df = test.get_store_items()

C:\Users\sundui\AppData\Local\Temp\ipykernel_3116\774024082.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(


17 0 KFC-БИЗНЕСС ПЛАЗА /Төв номын сан/
categories [<selenium.webdriver.remote.webelement.WebElement (session="b6299dd5849831e6bf298302cacf6c2b", element="9fb4dfc9-fc07-404e-9fab-fecde971ebbe")>, <selenium.webdriver.remote.webelement.WebElement (session="b6299dd5849831e6bf298302cacf6c2b", element="badbb71e-e0da-4874-bea9-f69ce7474e36")>, <selenium.webdriver.remote.webelement.WebElement (session="b6299dd5849831e6bf298302cacf6c2b", element="44d461e2-394c-4390-bc72-03156f138e05")>, <selenium.webdriver.remote.webelement.WebElement (session="b6299dd5849831e6bf298302cacf6c2b", element="e74df7d9-8265-4f3a-a950-93cb23fc4f41")>, <selenium.webdriver.remote.webelement.WebElement (session="b6299dd5849831e6bf298302cacf6c2b", element="122bdbe4-5912-41ea-a3d6-a988a0f11b25")>, <selenium.webdriver.remote.webelement.WebElement (session="b6299dd5849831e6bf298302cacf6c2b", element="242037ca-ca66-4b2a-9ec0-a88b97e7803f")>, <selenium.webdriver.remote.webelement.WebElement (session="b6299dd5849831e6bf298302ca

C:\Users\sundui\AppData\Local\Temp\ipykernel_3116\774024082.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(


17 1 KFC-Смарт /Өргөө кино театрын эсрэг талд/
categories [<selenium.webdriver.remote.webelement.WebElement (session="f1fee496ec2d6da432c81b48207b25d0", element="1c4cbef0-e9bd-4308-ad58-7ab72208ff0e")>, <selenium.webdriver.remote.webelement.WebElement (session="f1fee496ec2d6da432c81b48207b25d0", element="881dac6f-d15d-485f-bb80-e21cf665fe46")>, <selenium.webdriver.remote.webelement.WebElement (session="f1fee496ec2d6da432c81b48207b25d0", element="57df9632-bf1d-4087-91a3-254a6ec2b39d")>, <selenium.webdriver.remote.webelement.WebElement (session="f1fee496ec2d6da432c81b48207b25d0", element="a2c26733-7e1d-4018-9f93-292257f3860e")>, <selenium.webdriver.remote.webelement.WebElement (session="f1fee496ec2d6da432c81b48207b25d0", element="356902ab-3336-4f08-baa3-d64347ebb240")>, <selenium.webdriver.remote.webelement.WebElement (session="f1fee496ec2d6da432c81b48207b25d0", element="9b287b9a-c0d9-4c24-a2e6-a34b2befa71c")>, <selenium.webdriver.remote.webelement.WebElement (session="f1fee496ec2d6da432

C:\Users\sundui\AppData\Local\Temp\ipykernel_3116\774024082.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(


...

C:\Users\sundui\AppData\Local\Temp\ipykernel_3116\774024082.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(


17 10 KFC-Хоум Плаза /Хан-Уул/ - 24 цагийн салбар
categories [<selenium.webdriver.remote.webelement.WebElement (session="67c3c50317b8755fbe07699d4e9f24f9", element="cdff8449-6509-48db-85c2-361a761a4b96")>, <selenium.webdriver.remote.webelement.WebElement (session="67c3c50317b8755fbe07699d4e9f24f9", element="bd4b4b55-fe2f-41d5-992a-928937cc4dd2")>, <selenium.webdriver.remote.webelement.WebElement (session="67c3c50317b8755fbe07699d4e9f24f9", element="60769efc-1a74-4d4c-8bec-185615e8fdf8")>, <selenium.webdriver.remote.webelement.WebElement (session="67c3c50317b8755fbe07699d4e9f24f9", element="35027c19-fc2a-454d-bd57-71d371cd3112")>, <selenium.webdriver.remote.webelement.WebElement (session="67c3c50317b8755fbe07699d4e9f24f9", element="f3ca6567-9c9b-4c00-b511-73c9a3c52d3f")>, <selenium.webdriver.remote.webelement.WebElement (session="67c3c50317b8755fbe07699d4e9f24f9", element="0428331d-3873-4242-947f-d33d67301233")>, <selenium.webdriver.remote.webelement.WebElement (session="67c3c50317b8755

C:\Users\sundui\AppData\Local\Temp\ipykernel_3116\774024082.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(


17 11 KFC-ГАЛЛЕРИА /Галлериа моол 1 давхарт/
categories [<selenium.webdriver.remote.webelement.WebElement (session="7db3e4d126ff1ce61dbb37a47359efc8", element="77b88130-e5a9-432d-afb2-b56981631962")>, <selenium.webdriver.remote.webelement.WebElement (session="7db3e4d126ff1ce61dbb37a47359efc8", element="b2fa34c6-3f70-42be-9e7a-c308b9814d95")>, <selenium.webdriver.remote.webelement.WebElement (session="7db3e4d126ff1ce61dbb37a47359efc8", element="23bba922-d67e-4026-9af1-4ec53a4f74c7")>, <selenium.webdriver.remote.webelement.WebElement (session="7db3e4d126ff1ce61dbb37a47359efc8", element="fb96d53c-a455-4a16-8ac9-ba59b70ff4bf")>, <selenium.webdriver.remote.webelement.WebElement (session="7db3e4d126ff1ce61dbb37a47359efc8", element="7a917bd1-da87-40df-b770-42113f6c574c")>, <selenium.webdriver.remote.webelement.WebElement (session="7db3e4d126ff1ce61dbb37a47359efc8", element="3e0e6e72-581e-49ed-8d48-414fced8005a")>, <selenium.webdriver.remote.webelement.WebElement (session="7db3e4d126ff1ce61dbb

C:\Users\sundui\AppData\Local\Temp\ipykernel_3116\774024082.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(


In [ ]:
test_1 = list(df['store'].unique())

In [ ]:
test_2 = test.store_list_dict

In [ ]:
test_3 = [i for i in test_2.values()]

In [ ]:
def count(arr, n):
    mp = dict()
    non_dup = []
    
    for i in range(n):
        if arr[i] in mp.keys():
            mp[arr[i]] += 1
        else:
            mp[arr[i]] = 1
    for x in mp:
        if mp[x] == 1:
            non_dup.append(x)
            
    if len(non_dup) > 0:
        print(non_dup)
    else:
        print('no non duplicates')

In [ ]:
arr = test_1 + test_3
count(arr, len(arr))

In [ ]:
df